# Module 11 - Programming Assignment


## Reinforcement Learning with Value Iteration

These are the same maps from Module 1 but the "physics" of the world have changed. In Module 1, the world was deterministic. When the agent moved "south", it went "south". When it moved "east", it went "east". Now, the agent only succeeds in going where it wants to go *sometimes*. There is a probability distribution over the possible states so that when the agent moves "south", there is a small probability that it will go "east", "north", or "west" instead and have to move from there.

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in video games that use formal planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty in the actions. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do in **every state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

We have the same costs as before. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [105]:
costs = { '.': -1, '*': -3, '^': -5, '~': -7}
costs

{'.': -1, '*': -3, '^': -5, '~': -7}

and a list of offsets for `cardinal_moves`. You'll need to work this into your **actions**, A, parameter.

In [106]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

For Value Iteration, we require knowledge of the *transition* function, as a probability distribution.

The transition function, T, for this problem is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if the agent selects "north" then 70% of the time, it will go "north" but 10% of the time it will go "east", 10% of the time it will go "west", and 10% of the time it will go "south". If agent is at the edge of the map, it simply bounces back to the current state.

You need to implement `value_iteration()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different than A\* search which only returns actions to take from the start to the goal. This also explains why reinforcement learning doesn't take a `start` state.

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Use "x" for any mountain or other impassable square. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output). Please print it out exactly as requested: **NO EXTRA SPACES OR LINES**.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy? What are the problems with this policy as it is?


In [107]:
from typing import List, Tuple, Dict, Callable
import numpy as np
import copy

In [108]:
def read_world(filename):
    result = []
    with open(filename) as f:
        for line in f.readlines():
            if len(line) > 0:
                result.append(list(line.strip()))
    return result

<a id="max_of"></a>
## max_of

finds the maximium value of the difference between V and V_last **Used by**: [value_iteration](#value_iteration)


* **v_new List[List[str]]:** a matrix of max(R(s, a))
* **v_last dict:** a copy of matrix of max(R(s, a)) from the prior iteration 

* **returns int:** max of the difference between the current state and prior state

In [116]:
def max_of(v_new: List[List[int]], v_last: List[List[int]]):
    v= np.array(v_new)
    v_l= np.array(v_last)
    return np.max( v-v_l)

In [117]:
assert max_of([[1,1,5],[1,1,1],[1,1,1]], [[1,1,1],[1,1,1],[1,1,1]]) == 4
assert max_of([[1,1,6],[1,1,1],[1,1,1]], [[1,1,1],[1,1,1],[1,1,1]]) == 5
assert max_of([[1,1,7],[1,1,1],[1,1,1]], [[1,1,1],[1,1,1],[1,1,1]]) == 6

<a id="reward"></a>
## reward

finds the reward/cost given the state **Used by**: [value_iteration](#value_iteration)

* **state Tuple[int,int]:** the cords in the world
* **action Tuple[int, int]:** the offset move in the world
* **world List[List[str]]:** a map of the terrain
* **costs dict:** costs of terrain elements
* **goal Tuple (int, int):** The goal state
* **reward int:** the reward for achieving the goal state

* **returns Tuple:** successor state

In [118]:
def reward(state: Tuple[int,int], action: Tuple[int,int], world: List[List[str]], costs_: dict, goal_: Tuple[int, int], rewards_: int):
    if state == goal_:
        return rewards_
    else:
        return costs_[world[state[0]][state[1]]]

In [119]:
small_world = read_world( "small.txt")
assert reward((0,0), (0,1), small_world, costs, (3,3), 15) == -1
assert reward((1,1), (0,1), small_world, costs, (3,3), 15) == -3
assert reward((3,3), (0,1), small_world, costs, (3,3), 15) == 15

<a id="seccessor_state"></a>
## seccessor_state

finds the successor state given the state/action/world **Used by**: [value_iteration](#value_iteration)

* **state Tuple[int,int]:** the cords in the world
* **action Tuple[int, int]:** the offset move in the world
* **world List[List[str]]:** the grid world

* **returns Tuple[int, int]:** the resulting new state

In [120]:
def seccessor_state(state: Tuple[int,int], action: Tuple[int,int], world: List[List[str]]):
    assert state[0] in range(len(world))
    assert state[1] in range(len(world[0]))
    result = tuple(a+b for a,b in zip(state,action))
    if result[0] not in range(len(world)):
        return state
    elif result[1] not in range(len(world[0])):
        return state
    else:
        return result

In [121]:
assert seccessor_state((1,1), (1,2), small_world) == (2,3)
assert seccessor_state((0,0), (1,2), small_world) == (1,2)
assert seccessor_state((6,5), (1,2), small_world) == (6,5)

<a id="value_iteration"></a>
## value_iteration

performs policy search by iterating over the states, actions and meauring costs/rewards and discounted costs from successor states. Finds the best polcy/plan given the goal/reward

* **world List[List[str]]:** a map of the terrain
* **costs dict:** costs of terrain elements
* **goal Tuple (int, int):** The goal state
* **reward int:** the reward for achieving the goal state
* **actions List[]:** list of possible actions, A, as offsets.
* **gamma float:** the discount rate

* **returns dict:** a dict of policy

In [130]:
def value_iteration(world: List[List[str]], costs: dict, goal: Tuple[int, int], rewards: int, actions: List[Tuple[int,int]], gamma: float):
    V =[[0 for s in range(len(world[0]))] for s in range(len(world))] # t-1 values 
    t=0 
    while True:
        t= t+1  
        V_last = copy.copy(V)
        for s in [(i,j) for i in range(len(world)) for j in range(len(world[0]))]:
            Q={}
            for a in actions:
                Q[s,a] = reward(s, a, world, costs, goal, rewards)+ gamma*V_last[seccessor_state(s,a, world)[0]][seccessor_state(s,a, world)[1]] # reward of state/action and successor state discounted reward
            pi[s]= argmax(Q[s,a]) # store best action 
            V[s] = Q[s, pi[s]] # store reward of best action
        if max_of(V,V_last) < .1: # measure 
            return pi
            break

<a id="pretty_print_policy"></a>
## pretty_print_policy


* **cols int:** column dimension of the world
* **rows int:** row dimension of the world
* **policy dict:** The offline learned policy
* **goal Tuple(int,int):** The position of the goal in the grid world


* **returns str:** a printed map of policy across all states of the world

In [127]:
def pretty_print_policy( cols, rows, policy, goal):
    for c in range(cols):
        for r in range(rows):
            print(policy[r,c])
        


## Value Iteration

### Small World

In [131]:
small_world = read_world( "small.txt")
len(small_world[0])


6

In [132]:
goal = (len(small_world[0])-1, len(small_world)-1)
gamma = 0.9

small_policy = value_iteration(small_world, costs, goal, reward, cardinal_moves, gamma)

NameError: name 'Q' is not defined

In [ ]:
cols = len(small_world[0])
rows = len(small_world)

pretty_print_policy(cols, rows, test_policy, goal)

### Large World

In [ ]:
large_world = read_world( "large.txt")

In [ ]:
goal = (len(large_world[0])-1, len(large_world)-1) # Lower Right Corner FILL ME IN
gamma = 0.9

large_policy = value_iteration(large_world, costs, goal, reward, cardinal_moves, gamma)

In [ ]:
cols = len(large_world[0])
rows = len(large_world)

pretty_print_policy( cols, rows, large_policy, goal)